# Funciones API

In [1]:
import pandas as pd
import numpy as np
import json
import numpy as np

- def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [2]:
def PlayTimeGenre( genero : str ):
    data = pd.read_parquet('../dataset/genres_playtime.parquet')
    result = data[data['genres']==genero].reset_index()
    del data
    return {f"Año de lanzamiento con más horas jugadas para {genero}" : result['year'].loc[0]}

In [3]:
PlayTimeGenre('RPG')

{'Año de lanzamiento con más horas jugadas para RPG': 2011}

- def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [4]:
def UserForGenre( genero : str ):
    data = pd.read_parquet('../dataset/user_playtime2genres.parquet')
    data = data[data['genres']==genero]
    usuario = data.groupby(['user_id'])['playtime_forever'].sum().sort_values(ascending=False).reset_index()
    result = data[data['user_id'] == usuario['user_id'].loc[0]].reset_index()
    i = 0
    res = {f"Usuario con más horas jugadas para {genero}": usuario['user_id'].loc[0],"Horas jugadas":[]}
    while i<= len(result['user_id']) -1:
        res['Horas jugadas'].append({"Año":result['year'].loc[i], "Horas": result['playtime_forever'].loc[i]})
        i = i + 1
    del data
    del usuario
    del result
    return [res]

In [5]:
UserForGenre('RPG')

[{'Usuario con más horas jugadas para RPG': 'shinomegami',
  'Horas jugadas': [{'Año': 1999, 'Horas': 377},
   {'Año': 2000, 'Horas': 0},
   {'Año': 2001, 'Horas': 0},
   {'Año': 2002, 'Horas': 0},
   {'Año': 2003, 'Horas': 530943},
   {'Año': 2004, 'Horas': 696},
   {'Año': 2005, 'Horas': 185},
   {'Año': 2006, 'Horas': 2023},
   {'Año': 2007, 'Horas': 135020},
   {'Año': 2008, 'Horas': 1305},
   {'Año': 2009, 'Horas': 1382},
   {'Año': 2010, 'Horas': 4311},
   {'Año': 2011, 'Horas': 17752},
   {'Año': 2012, 'Horas': 12819},
   {'Año': 2013, 'Horas': 107775},
   {'Año': 2014, 'Horas': 81938},
   {'Año': 2015, 'Horas': 111419},
   {'Año': 2016, 'Horas': 15550},
   {'Año': 2017, 'Horas': 0}]}]

- def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [6]:
def UsersRecommend( año : int ):
    data = pd.read_parquet('../dataset/user_recommend.parquet')
    # Filtrar los datos para el año solicitado
    try:    
        filter = data[(data['year'] == año) & (data['recommend'] == True) & (data['sentiment_analysis'] > 0)]
        list = filter.groupby(by=['item_id']).size().sort_values(ascending=False)
        ids = list.index
        A = filter[filter['item_id'] == ids[0]]['title'].reset_index().loc[0]
        B = filter[filter['item_id'] == ids[1]]['title'].reset_index().loc[0]
        C = filter[filter['item_id'] == ids[2]]['title'].reset_index().loc[0]
        result = [{"Puesto 1" : A['title']},{"Puesto 2" : B['title']}, {"Puesto 3" : C['title']}]
        return result
    except IndexError:
        return [{f"No hay registros para el año {año}"}]

In [7]:
UsersRecommend(2015)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Team Fortress 2'},
 {'Puesto 3': "Garry's Mod"}]

- def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [8]:
def UsersNotRecommend( año : int ):
    data = pd.read_parquet('../dataset/user_recommend.parquet')
    # Filtrar los datos para el año solicitado
    try:    
        filter = data[(data['year'] == año) & (data['recommend'] == False) & (data['sentiment_analysis'] == 0)]
        list = filter.groupby(by=['item_id']).size().sort_values(ascending=False)
        ids = list.index
        A = filter[filter['item_id'] == ids[0]]['title'].reset_index().loc[0]
        B = filter[filter['item_id'] == ids[1]]['title'].reset_index().loc[0]
        C = filter[filter['item_id'] == ids[2]]['title'].reset_index().loc[0]
        result = [{"Puesto 1" : A['title']},{"Puesto 2" : B['title']}, {"Puesto 3" : C['title']}]
        return result
    except IndexError:
        return [{f"No hay registros para el año {año}"}]

In [9]:
UsersNotRecommend(2012)

[{'Puesto 1': 'Call of Duty®: Black Ops II'},
 {'Puesto 2': 'Resident Evil: Operation Raccoon City'},
 {'Puesto 3': "The Kings' Crusade"}]

- def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [10]:
def sentiment_analysis( año : int ):
    data = pd.read_parquet('../dataset/user_recommend.parquet')
    try:
        filter = data[data['year'] == año]
        group = filter.groupby(by=['sentiment_analysis']).size()
        return {"Negative =": group[0], "Neutral =": group[1], "Positive =": group[2]}
    except:
        return {"Error" : "No hay datos para el año ingresado"}

In [11]:
sentiment_analysis(2011)

{'Negative =': 49, 'Neutral =': 67, 'Positive =': 349}

- def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [12]:
#Funcion cosine_similarity
def similarityCosine(vector1, vector2):
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    if norm1 == 0 or norm2 == 0:
        return 0 
    else:
        return np.dot(vector1, vector2) / (norm1 * norm2)

In [13]:
def recomendacion_juego(id : int):
    reviews  = pd.read_parquet('../dataset/reviews_per_item.parquet')
    filtro = reviews[reviews['id'] == id]
    try:
        fila1 = filtro.iloc[0]
    except IndexError:
        return {"error": "No se encontraron datos para el ID del juego dado."}   
    fila1 = filtro.iloc[0]
    lista = []
    i = 0
    while i < len(reviews):
        fila2 = reviews.iloc[i]
        lista.append(similarityCosine(fila1,fila2))
        i += 1
    reviews['similarity_cosine'] = lista
    recomendacion =reviews[['id','similarity_cosine']].sort_values(by=['similarity_cosine'], ascending=False).head(10)
    del reviews
    rec = []
    j = 0
    while j < 6:
        if recomendacion['id'].iloc[j] != id:
            rec.append(recomendacion['id'].iloc[j])
        j+=1
    return {f"Los juegos recomendados para {id} son: {rec}"}

In [14]:
recomendacion_juego(43110)

{'Los juegos recomendados para 43110 son: [1250, 304500, 235400, 269170, 263200, 228760]'}